![](img/spark-logo.png)

> Apache Spark™ is a fast and general engine for large-scale data processing.

## ¿Por qué Spark? (una vez más)
Hadoop nació (~2005) para procesar grandes cantidades de datos en paralelo. Poco a poco
fueron surgiendo nuevas problemáticas que no se podían resolver con el paradigma *MapReduce* y fueron apareciendo
nuevos proyectos para solventar estas
problemáticas, siendo necesario así una "jungla" de programas para un trabajo de big data:

&nbsp;    
&nbsp;    
&nbsp;    


![](img/mapreduce_ecosystem.png)


&nbsp;    
&nbsp;    
&nbsp;    


Spark nace con dos finalidades claras: ser rápido, para ello cambia la manera de trabajar internamente (utiliza memoria, *RDD*,*DAG*...) y unificar
bajo un solo proyecto los grandes problemas de datos hasta el momento: Procesamiento en Batch,
en *streaming*, *machine learning*, *SQL*...   

Además incluye en el mismo proyecto varios lenguajes: Scala, Java, python y R.

&nbsp;    
&nbsp;     


<center>

<h1>¡NO SOLO JAVA!</h1>

![spark](img/esquema2.png)
</center>
&nbsp;    
&nbsp;    
&nbsp;    

Hoy poy hoy Spark es casi sinonimo de big data y está presente en la mayoria de proyectos siendo la primera opción para el procesamiento masivo de datos. Esto ha hecho que muchas de las 
aplicaciones ya existentes se hayan hecho compatibles con Spark y que estén surgiendo nuevas
enfocadas en trabajar con Spark.

&nbsp;    
&nbsp;    
&nbsp;    


![](img/ecosystem.png)

<h1 style="font-size:40px;"> Python + Spark = PySpark </h1>
![](img/pyspark.png)

Aunque `spark` está escrito en scala y es el principal lenguaje para trabajar con el. También están soportados otros lenguajes: Java, R y python. 

Aunque hay soporte para R este todavía es un poco limitado, y si queremos sacar el máximo provecho a spark desde un lenguaje habitual para el análisis de datos, python es la mejor opción a día de hoy.

Es más en las últimas versiones de spark se están inplementado nuevas caracteríticas únicas para python como las
[vectorized-udfs](https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html).


&nbsp;    
&nbsp;    

Desde las últimas versiones pyspark se puede usar instalar directamente con [pip](https://pypi.python.org/pypi/pyspark/2.2.0) y podemos consular la documentación [aquí](http://spark.apache.org/docs/latest/api/python/index.html). 

En nuestro cluster podemos acceder por consola con:

```
pyspark --version
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 1.6.0
      /_/
                        
Type --help for more information.
```

O a la versión 2 que es la que usaremos con:

```
pyspark2 --version
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.2.0.cloudera1
      /_/
                        
Using Scala version 2.11.8, Java HotSpot(TM) 64-Bit Server VM, 1.8.0_152
Branch HEAD
Compiled by user jenkins on 2017-07-13T00:28:58Z
Revision 39f5a2b89d29d5d420d88ce15c8c55e2b45aeb2e
Url git://github.mtv.cloudera.com/CDH/spark.git
Type --help for more information.

```

Para conectarnos con `spark` desde el notebook tenemos que configurar la conexión y usar el kernel `Anaconda2`:

![](img/kernel.png)

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [2]:
conf = (SparkConf().setAppName(u"[ICAI] Intro Pyspark")) #establece el nombre de la aplicacion de spark

In [3]:
spark = (

    SparkSession.builder
    .config(conf=conf)
    .enableHiveSupport()
    .getOrCreate()

)

La variable `spark` es el 'entry point' al framework spark y la que usaremos para interactuar con el cluster

In [ ]:
print(spark)

In [ ]:
print(spark.sparkContext)

### Word Count con `pyspark` (LA RUTA POR DEFECTO ES HDFS YA QUE SPARK TRABAJA SOBRE HADOOP Y ES DISTRIBUIDO)

In [ ]:
readme = spark.sparkContext.textFile('/datos/README.md') #lee un fichero de texto plano (LA RUTA ES HDFS)
#Tarda poco porque es lazy y no hace nada hasta que hay una accion

In [ ]:
type(readme)

In [ ]:
readme.take(1) #lee la primera fila (UNA LISTA DE PYTHON CON UN ELEMENTO DE TIPO STRING)

In [ ]:
readme.count()

In [ ]:
import re

### *Mirar diferencia entre flatmap y map

In [ ]:
(

    readme #es un rdd: UN RDD ES UNA LISTA DISTRIBUIDA

    # Divido cada por espacios (divide por cualquier caracter que sea una separacion)
    .flatMap(lambda x: re.split('\s+',x)) 
            #si pusieramos solo map nos daria una lista de listas (anidada) por cada fila
            #con flatmap todo esta al mismo nivel

    # Creo un pair RDD
    .map(lambda x: (x,1)) #FUNCION MAP DEVUELVE EL MISMO NUMERO DE ELEMENTOS QUE LA FUNCION DE ORIGEN

    # Reduzco por key y sumo los unos para contar
    .reduceByKey(lambda a, b: a + b)

    # Ordeno de mayor a menor el conteo
    .sortBy(lambda x: -x[1]) #ORDENAR POR EL SEGUNDO ELEMENTO X[1] DE MAYOR A MENOR (-)

).take(10)

### De python a spark

Podemos pasar variables de python directamente a *RDD's* y viceversa:

In [ ]:
import numpy as np

In [ ]:
colores = np.array(['blue', 'red', 'green', 'yellow', 'brown', 'black'])

In [ ]:
muestra = np.random.choice(colores, 1000000)

In [ ]:
len(muestra)

In [ ]:
muestra[:10]

In [ ]:
rdd_muestra = spark.sparkContext.parallelize(muestra, 10)

In [ ]:
rdd_muestra.count()

In [ ]:
rdd_muestra.take(10) #Siempre que haces un take devuekve una lista

In [ ]:
(

    rdd_muestra
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)

).collect()

### Arriba es como hacerlo con pyspark, abajo solo con python
Usamos pyspark cuando tenemos MUCHOS datos y no podemos trabajar en nuestro ordenador

In [ ]:
np.unique(muestra, return_counts=True) #No ponemos collect porque muestra ya es de python

¿Y si los datos no caben en memoría?

In [ ]:
gran_muestra = (

    spark.sparkContext
    # Usamos range como si fuera un bucle
    .range(10, numSlices=200)
    .flatMap(lambda _: np.random.choice(colores, 1000000))

)

In [ ]:
gran_muestra.take(10)

In [ ]:
print('%.2E' % (10 * 1000000))

In [ ]:
n = gran_muestra.count()
n
print('%.2E' % n)

In [ ]:
(

    gran_muestra
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)

).collect()

### <font color='darkred'> CUIDADO:

Al usar `collect` se recoge todo el *RDD* en el driver, así que hay que estar seguro de que el tamaño sea pequeño o tendremos problemas de memoria.

`
np.unique(gran_muestra.collect(),return_counts=True)
`
*Ponemos collect porque gran muestra es un RDD

### Leer ficheros del HDFS

In [ ]:
!hadoop fs -ls /datos/diamonds.csv

In [ ]:
!hadoop fs -text /datos/diamonds.csv | head

In [4]:
diamonds = spark.sparkContext.textFile('/datos/diamonds.csv') #SI NO TIRA HAY QUE DARLE A KERNEL/RESTART AND CLEAR OUTPUT 
        #Y VOLVER A LANZAR LA SESION DE SPARK

In [5]:
diamonds.take(10)

['"carat","cut","color","clarity","depth","table","price","x","y","z"',
 '0.23,"Ideal","E","SI2",61.5,55,326,3.95,3.98,2.43',
 '0.21,"Premium","E","SI1",59.8,61,326,3.89,3.84,2.31',
 '0.23,"Good","E","VS1",56.9,65,327,4.05,4.07,2.31',
 '0.29,"Premium","I","VS2",62.4,58,334,4.2,4.23,2.63',
 '0.31,"Good","J","SI2",63.3,58,335,4.34,4.35,2.75',
 '0.24,"Very Good","J","VVS2",62.8,57,336,3.94,3.96,2.48',
 '0.24,"Very Good","I","VVS1",62.3,57,336,3.95,3.98,2.47',
 '0.26,"Very Good","H","SI1",61.9,55,337,4.07,4.11,2.53',
 '0.22,"Fair","E","VS2",65.1,61,337,3.87,3.78,2.49']

In [6]:
filtrado = (

    diamonds
    .map(lambda x: x.split(','))
    .filter(lambda x: 'Fair' in x[1] and 'SI2' in x[3])

)

In [7]:
filtrado.count()

466

In [8]:
filtrado.take(2)

[['0.86',
  '"Fair"',
  '"E"',
  '"SI2"',
  '55.1',
  '69',
  '2757',
  '6.45',
  '6.33',
  '3.52'],
 ['0.96',
  '"Fair"',
  '"F"',
  '"SI2"',
  '66.3',
  '62',
  '2759',
  '6.27',
  '5.95',
  '4.07']]

### Transformaciones y Acciones

La API de Pyspark es muy parecida al core en *scala*. También tenemos transformaciones y acciones:    
&nbsp;   

<center>
![](img/RDD_Operations.png)
</center>

* https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations
* https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions

In [9]:
peliculas = [
    'http://www.imdb.com/title/tt0071562',
    'http://www.imdb.com/title/tt0110912',
    'http://www.imdb.com/title/tt0050083',
    'http://www.imdb.com/title/tt0108052',
    'http://www.imdb.com/title/tt0468569',
    'http://www.imdb.com/title/tt0068646',
    'http://www.imdb.com/title/tt0167260',
    'http://www.imdb.com/title/tt0060196',
    'http://www.imdb.com/title/tt0137523',
    'http://www.imdb.com/title/tt0111161',
]

In [10]:
import requests
from bs4 import BeautifulSoup

In [11]:
def parsear_html(texto):
    soup = BeautifulSoup(texto,'lxml')
    item = dict()
    item['titulo'] = soup.find("h1").find(text=True).replace(u'\xa0',' ').strip()
    item['ratingvalue'] = float(soup.select_one('[itemprop="ratingValue"]').text)
    return item

In [12]:
descargas = (
    
    spark.sparkContext
    .parallelize(peliculas)
    .map(lambda x: requests.get(x).content)

)

In [13]:
descargas.count()

10

In [14]:
parseados = descargas.map(parsear_html)

In [15]:
parseados.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 4.0 failed 4 times, most recent failure: Lost task 1.3 in stage 4.0 (TID 12, worker03.bigdata.alumnos.upcont.es, executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/cloudera/parcels/SPARK2/lib/spark2/python/pyspark/worker.py", line 253, in main
    process()
  File "/opt/cloudera/parcels/SPARK2/lib/spark2/python/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/cloudera/parcels/SPARK2/lib/spark2/python/pyspark/serializers.py", line 379, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/cloudera/parcels/SPARK2/lib/spark2/python/pyspark/util.py", line 55, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-11-b7c2678cd2d0>", line 5, in parsear

### Cache

Podemos cachear un *RDD*, para no tener que recalcuarlo cada vez. Muy útil cuando estamos explorando los datos o tenemos que hacer dos acciones distintas sobre el mismo *RDD*.

In [16]:
%%timeit
descargas.count()

1.21 s ± 92.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
parseados.collect()

In [ ]:
descargas.cache() #lo GUARDA EN EL CACHE PARA QUE LA PROXIMA VEZ NO TENGAMOS QUE CARGARLO EN MEMORIA

In [ ]:
%%timeit
descargas.count()

In [ ]:
parseados = descargas.map(parsear_html)

In [ ]:
%%timeit
parseados.collect()

In [ ]:
descargas.is_cached

In [ ]:
descargas.unpersist()

In [ ]:
descargas.is_cached

####  Al finalizar, siempre hay que cerrar la conexión de spark:

In [ ]:
spark.stop()